# funciones para la API

In [18]:
# importamos librerias
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

cargamos los archivos parquet

In [19]:
# cargamos dataframes 
steam_games= pd.read_parquet('data/clean_steam_games.parquet')
user_reviews= pd.read_parquet('data/clean_user_reviews.parquet')
user_items= pd.read_parquet('data/clean_user_items.parquet')

# 1- def developer( desarrollador : str ): Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora. Ejemplo de retorno:

dataframe hecho para la funcion

In [20]:
# extraemos las columnas del dataframe steam_games que vamos a necesitar
funcion1=steam_games[['item_id', 'price', 'developer', 'release_year']]
# eliminamos duplicados
funcion1= funcion1.drop_duplicates()
funcion1.head()

,item_id,price,developer,release_year
0,761140,4.99,Kotoshiro,2018
5,643980,0.00,Secret Level SRL,2018
9,670290,0.00,Poolians.com,2017
14,767400,0.99,彼岸领域,2017
17,772540,3.99,Trickjump Games Ltd,2018


In [28]:
def developer_alternativo(desarrollador):
    # filtramos dataframe para que solo devuelva la informacion del desarrollador dado
    df_filtrado = funcion1[funcion1['developer'] == desarrollador].copy()

    # agrupamos los datos
    agrupado = df_filtrado.groupby('release_year').agg(
        Cantidad_de_Items=pd.NamedAgg(column='item_id', aggfunc='count'),
        Contenido_Free=pd.NamedAgg(column='price', aggfunc=lambda x: (x == 0).sum())
    )

    # calculamos el porcentaje de los juegos de contenido gratuito
    agrupado['Porcentaje_Contenido_Free'] = (agrupado['Contenido_Free'] / agrupado['Cantidad_de_Items']) * 100

    # formateamos el dataFrame 
    agrupado.reset_index(inplace=True)
    agrupado.rename(columns={'release_year': 'Año'}, inplace=True)
    agrupado['Porcentaje_Contenido_Free'] = agrupado['Porcentaje_Contenido_Free'].map('{:.2f}%'.format)
    return agrupado[['Año', 'Cantidad_de_Items', 'Porcentaje_Contenido_Free']].to_dict(orient='records')


developer_alternativo('Valve')


[{'Año': 1998, 'Cantidad_de_Items': 1, 'Porcentaje_Contenido_Free': '0.00%'},
 {'Año': 1999, 'Cantidad_de_Items': 1, 'Porcentaje_Contenido_Free': '0.00%'},
 {'Año': 2000, 'Cantidad_de_Items': 2, 'Porcentaje_Contenido_Free': '0.00%'},
 {'Año': 2001, 'Cantidad_de_Items': 1, 'Porcentaje_Contenido_Free': '0.00%'},
 {'Año': 2003, 'Cantidad_de_Items': 1, 'Porcentaje_Contenido_Free': '0.00%'},
 {'Año': 2004, 'Cantidad_de_Items': 5, 'Porcentaje_Contenido_Free': '0.00%'},
 {'Año': 2006, 'Cantidad_de_Items': 2, 'Porcentaje_Contenido_Free': '0.00%'},
 {'Año': 2007, 'Cantidad_de_Items': 3, 'Porcentaje_Contenido_Free': '33.33%'},
 {'Año': 2008, 'Cantidad_de_Items': 1, 'Porcentaje_Contenido_Free': '0.00%'},
 {'Año': 2009, 'Cantidad_de_Items': 1, 'Porcentaje_Contenido_Free': '0.00%'},
 {'Año': 2010, 'Cantidad_de_Items': 2, 'Porcentaje_Contenido_Free': '50.00%'},
 {'Año': 2011, 'Cantidad_de_Items': 1, 'Porcentaje_Contenido_Free': '0.00%'},
 {'Año': 2012, 'Cantidad_de_Items': 1, 'Porcentaje_Contenido_F

# 2- def userdata(user_id:str): debe devolver la cantidad de dinero gastado por el usuario, el porcentaje de recomendacion en base a reviews.recommend y cantidad de items

dataframe hecho para la funcion

In [8]:
# extraemos las columna del dataframe steam_games que vamos a necesitar para la funcion
data_games=steam_games[['item_id', 'genres', 'price']]
# extraemos las columnas del dataframe user_reviews que vamos a necesitar para la funcion
data_reviews= user_reviews[['item_id','user_id', 'recommend']]
# con los datos recogidos se crea el dataframe con el que vamos a correr la funcion
funcion2= data_reviews.merge(data_games, on='item_id')
# eliminamos los duplicados
funcion2= funcion2.drop_duplicates()
# reiniciamos los indices del dataframe
funcion2.reset_index(drop=True, inplace=True)
funcion2.head()

,item_id,user_id,recommend,genres,price
0,1250,76561197970982479,True,Action,19.99
1,1250,death-hunter,True,Action,19.99
2,1250,DJKamBer,True,Action,19.99
3,1250,diego9031,True,Action,19.99
4,1250,76561198081962345,True,Action,19.99


In [9]:
def userdata(user_id:str):
    # filtramos el dataframe para obtener solo la informacion del usuario dado 
    df_filtrado = funcion2.loc[funcion2["user_id"]== user_id]
    # contamos la cantidad de items unicos por usuario
    total_items= df_filtrado['item_id'].nunique()
    # calculamos el porcentaje de recomendacion
    porcentaje_recomendacion= round((df_filtrado['recommend'].sum() / total_items) * 100, 2)
    # calculamos la cantidad de dinero que el usuario ha gastado en items
    cantidad_dinero= round((df_filtrado['price'].sum()), 4)
    return {f'usuario':user_id, 'dinero gastado':f'{cantidad_dinero} USD', 'porcentaje de recomendacion':f'{porcentaje_recomendacion}%', 'cantidad de items':total_items}

userdata('snakedog')

{'usuario': 'snakedog',
 'dinero gastado': '84.93 USD',
 'porcentaje de recomendacion': '277.78%',
 'cantidad de items': 9}

# 3- def UserForGenre(genero:str): debe devolver el usuario que acumula mas horas jugadas para el genero dado y una lista de la acumulacion de horas jugadas por año de lanzamiento

dataframe hecho para la funcion

In [10]:
# extraemos las columnas del dataframe steam_games que vamos a necesitar para la funcion
data_games=steam_games[['item_id', 'genres', 'release_year']]
# extraemos las columnas del dataframe user_items que vamos a necesitar para la funcion
data_items= user_items[['item_id','user_id', 'playtime_forever']]
# con los datos extraidos se crea el dataframe con el que vamos a correr la funcion
funcion3= data_items.merge(data_games, on='item_id')
# eliminamos columna que no vamos a necesitar
funcion3= funcion3.drop('item_id', axis=1)
# eliminamos duplicados
funcion3= funcion3.drop_duplicates()
# reseteamos indices
funcion3.reset_index(drop=True, inplace=True)
funcion3.head()

,user_id,playtime_forever,genres,release_year
0,76561197970982479,6.0,Action,2000
1,76561198011743390,93.0,Action,2000
2,76561197967794920,108.0,Action,2000
3,MyCuntDotComDotAu,328.0,Action,2000
4,76561198019743758,580.0,Action,2000


In [11]:
# cambiamos tipo de dato a int
funcion3['playtime_forever']= funcion3['playtime_forever'].astype(int)

In [12]:
def UserForGenreee(genero:str):
  # filtramos dataframe para obtener la informacion del genero especificado
  generos = funcion3[funcion3['genres'].str.contains(genero)]

  # encontramos al usuario con mas horas jugadas aplicando funcion lambda
  usuario_max_horas = generos.assign(
    total_horas=lambda x: x['playtime_forever'].groupby(x['user_id']).transform('sum')
  ).sort_values('total_horas', ascending=False).iloc[0]['user_id']

  # Obtenemos las horas jugadas por año
  horas_por_año = generos.groupby('release_year')['playtime_forever'].sum().reset_index(name='Horas')
  horas_por_año= horas_por_año.rename(columns={'release_year':'Año'})
  # Convertir a una lista de diccionarios
  horas_lista = horas_por_año.to_dict('records')

  
  return {
    "Usuario con más horas jugadas para Género " + genero: usuario_max_horas,
    "Horas jugadas": horas_lista
  }
UserForGenreee('Adventure')

{'Usuario con más horas jugadas para Género Adventure': 'NooodleBox',
 'Horas jugadas': [{'Año': 1983, 'Horas': 3},
  {'Año': 1984, 'Horas': 12},
  {'Año': 1987, 'Horas': 711},
  {'Año': 1988, 'Horas': 509},
  {'Año': 1989, 'Horas': 32},
  {'Año': 1992, 'Horas': 150},
  {'Año': 1993, 'Horas': 269},
  {'Año': 1994, 'Horas': 41},
  {'Año': 1996, 'Horas': 1564},
  {'Año': 1997, 'Horas': 4780},
  {'Año': 1998, 'Horas': 766},
  {'Año': 1999, 'Horas': 1170},
  {'Año': 2001, 'Horas': 195},
  {'Año': 2002, 'Horas': 530},
  {'Año': 2003, 'Horas': 79598},
  {'Año': 2004, 'Horas': 1378},
  {'Año': 2005, 'Horas': 883},
  {'Año': 2006, 'Horas': 3161},
  {'Año': 2007, 'Horas': 6266},
  {'Año': 2008, 'Horas': 54077},
  {'Año': 2009, 'Horas': 79782},
  {'Año': 2010, 'Horas': 380808},
  {'Año': 2011, 'Horas': 3902786},
  {'Año': 2012, 'Horas': 1001606},
  {'Año': 2013, 'Horas': 2597554},
  {'Año': 2014, 'Horas': 1253938},
  {'Año': 2015, 'Horas': 3698037},
  {'Año': 2016, 'Horas': 1200314},
  {'Año': 2

# 4- def best_developer_year(anio:int): devuelve el top 3 de los desarrolladores con juegos mas recomendados por usuarios para el anio dado. (reviews.recommend= True y comentarios positivos)

dataframe hecho para la funcion

In [13]:
# extraemos las columna del dataframe steam_games que vamos a necesitar para la funcion
data_games= steam_games[['item_id', 'developer', 'release_year']]
# extraemos las columnas del dataframe user_reviews que vamos a necesitar para la funcion
data_reviews= user_reviews[['item_id', 'sentiment_analysis', 'recommend']]
# con los datos escogidos se crea el dataframe con el que vamos a correr la funcion
funcion4= data_reviews.merge(data_games, on='item_id')
# eliminamos columna que no vamos a necesitar
funcion4= funcion4.drop('item_id', axis=1)
# eliminamos los duplicados
funcion4= funcion4.drop_duplicates()
# reseteamos los indices
funcion4.reset_index(drop=True, inplace=True)
funcion4.head()


,sentiment_analysis,recommend,developer,release_year
0,2,True,Tripwire Interactive,2009
1,1,True,Tripwire Interactive,2009
2,0,True,Tripwire Interactive,2009
3,0,False,Tripwire Interactive,2009
4,2,False,Tripwire Interactive,2009


In [14]:
def best_developer_year(year: int):
    # filtramos dataframe para obtener solo las recomendaciones del año dado
    df_filtrado= funcion4[(funcion4['release_year']== year) & (funcion4['recommend']==True)]
    # contamos las recomendaciones por desarrollador
    conteo_recomendaciones= df_filtrado['developer'].value_counts()
    # obtenemos top 3 de los desarrolladores
    top_3_best_developers= conteo_recomendaciones.nlargest(3).index.tolist()
    return [{'Puesto 1':top_3_best_developers[0], 'Puesto 2':top_3_best_developers[1], 'Puesto 3':top_3_best_developers[2]}]
best_developer_year(2011)

[{'Puesto 1': '2K Czech,Feral Interactive (Mac)',
  'Puesto 2': 'id Software',
  'Puesto 3': 'Eden Studios'}]

# 5- def developer_reviews_analysis( desarrolladora : str ): Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.

dataframe hecho para la funcion

In [30]:
# extraemos las columna del dataframe steam_games que vamos a necesitar para la funcion
data_games=steam_games[['item_id','developer','release_year']]
# extraemos las columnas del dataframe user_reviews que vamos a necesitar para la funcion
data_reviews= user_reviews[['item_id', 'sentiment_analysis', 'review']]
# con los datos escogidos se crea el dataframe con el que vamos a correr la funcion
funcion5= data_reviews.merge(data_games, on='item_id')
# eliminamos columna que no vamos a necesitar
funcion5= funcion5.drop('item_id', axis=1)
# eliminamos los duplicados
funcion5= funcion5.drop_duplicates()
# reseteamos los indices
funcion5.reset_index(drop=True, inplace=True)
funcion5.head()


,sentiment_analysis,review,developer,release_year
0,2,Simple yet with great replayability. In my opi...,Tripwire Interactive,2009
1,2,"Amazing, Non-stop action of blowing stuff to b...",Tripwire Interactive,2009
2,2,"Compared to Left 4 Dead 2, this game REALLY gi...",Tripwire Interactive,2009
3,1,Jogo ♥♥♥♥.,Tripwire Interactive,2009
4,1,cara nas imagens esse jogo da pouco de medo ma...,Tripwire Interactive,2009


In [31]:
def developer_reviews_analysis(desarrolladora:str):
    # filtramos dataframe para obtener solo la informacion del desarrollador dado
    df_filtrado= funcion5[funcion5['developer']==desarrolladora]
    # contamos la cantidad de reseñas para cada tipo de sentimiento 
    sentiment_count = df_filtrado['sentiment_analysis'].value_counts().to_dict()
    # resultado a devolver
    resultado= {desarrolladora: ['Negativo: ' + str(sentiment_count.get(0,0)), 'Positivo:' + str(sentiment_count.get(2,0))]}
    return resultado

developer_reviews_analysis('Valve')

{'Valve': ['Negativo: 1545', 'Positivo:4736']}

# guardamos archivos

guardamos los archivos en formato parquet para cada una de las funciones creadas


In [29]:
funcion1.to_parquet('data/developer.parquet')
funcion2.to_parquet('data/userdata.parquet')
funcion3.to_parquet('data/UserForGenre.parquet')
funcion4.to_parquet('data/best_developer_year.parquet')
funcion5.to_parquet('data/developer_reviews_analysis.parquet')